In [1]:
import san
from src_end2end import statistical_features
import lsa_features
import pickle
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
import skopt
from skopt import gp_minimize
from sklearn import preprocessing
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from sklearn.metrics import f1_score
st_models = ["roberta-large-nli-stsb-mean-tokens", "xlm-r-large-en-ko-nli-ststb", "distilbert-base-nli-mean-tokens"]

In [2]:
from sentence_transformers import SentenceTransformer
st_models = ["roberta-large-nli-stsb-mean-tokens", "xlm-r-large-en-ko-nli-ststb", "distilbert-base-nli-mean-tokens"]
def embedd_bert(text, st_model = 'paraphrase-distilroberta-base-v1', split = 'train'):    
    paths = "temp_berts/"+st_model+"_"+split+'.pkl'
    if os.path.isfile(paths):
        sentence_embeddings = pickle.load(open(paths,'rb')) 
        return sentence_embeddings
    model = SentenceTransformer(st_model)
    sentence_embeddings = model.encode(text)
    with open(paths, 'wb') as f:
        pickle.dump(sentence_embeddings, f)
    return sentence_embeddings

In [3]:
from sentence_transformers import SentenceTransformer
st_models = ["roberta-large-nli-stsb-mean-tokens", "xlm-r-large-en-ko-nli-ststb", "distilbert-base-nli-mean-tokens"]
def embedd_bert2(text, st_model = 'paraphrase-distilroberta-base-v1'):    
    text = [t[:512] for t in text]
    model = SentenceTransformer(st_model)
    sentence_embeddings = model.encode(text)
    return sentence_embeddings

In [19]:
def export_kgs(dataset):
    path = "representations/"+dataset+"/"
    for split in ["train", "dev", "test"]:
        for kg in ["complex", "transe", "quate", "simple", "rotate", "distmult"]:
            path_tmp = path + split + "/" + kg + ".csv"
            tmp_kg = prep_kgs(kg, split)
            tmp_kg = np.array((tmp_kg))
            np.savetxt(path_tmp, tmp_kg, delimiter=",")
        

In [18]:
def prep_kgs(kg_emb, split='train'):
    embs = []
    global dataset
    path_in = "kg_emb_dump/"+dataset+"/"+split+"_"+kg_emb+'_n.pkl'
    with open(path_in, "rb") as f:
        kgs_p = pickle.load(f)
    for x,y in kgs_p:
        embs.append(y)
    return embs

In [5]:
def export_kgs_spec(dataset):
    path = "representations/"+dataset+"/"
    for split in ["train", "dev", "test"]:
        for kg in ["complex", "transe", "quate", "simple", "rotate", "distmult"]:
            path_tmp = path + split + "/" + kg + "_entity.csv"
            tmp_kg = prep_kgs2(kg, split)
            tmp_kg = np.array((tmp_kg))
            np.savetxt(path_tmp, tmp_kg, delimiter=",")
        

In [7]:
def export_LM(dataset):
    texts = {}
    ys = {}
    path = "representations/"+dataset+"/"
    for thing in ["train", "dev", "test"]:
        path_in = "data/final/"+dataset+"/"+thing+'.csv'
        df = pd.read_csv(path_in, encoding='utf-8')
        texts[thing] = df.text_a.to_list()
        #ys[thing] = df.label.to_list()
        staticstical = statistical_features.fit_space(texts[thing])
        kg = 'stat'
        path_tmp = path + thing + "/" + kg + ".csv"
        np.savetxt(path_tmp, staticstical, delimiter=",")
        
        bertz = embedd_bert2(texts[thing], st_models[0])
        kg = st_models[0]
        path_tmp = path + thing + "/" + kg + ".csv"
        np.savetxt(path_tmp, bertz, delimiter=",")
        
        bertz2 = embedd_bert2(texts[thing], st_models[1]) 
        kg = st_models[1]
        path_tmp = path + thing + "/" + kg + ".csv"
        np.savetxt(path_tmp, bertz2, delimiter=",")
         
        bertz3 = embedd_bert2(texts[thing], st_models[2]) 
        kg = st_models[2]
        path_tmp = path + thing + "/" + kg + ".csv"
        np.savetxt(path_tmp, bertz3, delimiter=",")       

In [25]:
for dataset in tqdm(["pan2020", "AAAI2021_COVID19_fake_news", "LIAR_PANTS", "ISOT", "FakeNewsNet"]):
    path = "representations/"+dataset+"/"
    for thing in ["train", "dev", "test"]:
        path_in = "data/final/"+dataset+"/"+thing+'.csv'
        df = pd.read_csv(path_in, encoding='utf-8')
        if dataset == "pan2020":
            ys = df.labels.to_list()
        else:
            ys = df.label.to_list()            
        path_tmp = path + thing + "/" + "_ys.csv"
        np.savetxt(path_tmp, ys, delimiter=",")

100%|██████████| 5/5 [00:01<00:00,  3.47it/s]


In [12]:
def prep_kgs2(kg_emb, split='train'):
    embs = []
    global dataset
    path_in = "kg_emb_dump/"+dataset+"/"+split+"_"+kg_emb+'_speakers.pkl'
    with open(path_in, "rb") as f:
        kgs_p = pickle.load(f)
    for x,y in kgs_p:
        embs.append(y)
    return embs

In [13]:
from tqdm import tqdm
for dataset in tqdm(["LIAR_PANTS", "FakeNewsNet"]):
    export_kgs_spec(dataset)

100%|██████████| 2/2 [01:11<00:00, 35.73s/it]


In [20]:
for dataset in tqdm(["pan2020", "AAAI2021_COVID19_fake_news", "LIAR_PANTS", "ISOT", "FakeNewsNet"]):
    export_kgs(dataset)

100%|██████████| 5/5 [04:07<00:00, 49.49s/it]


In [ ]:
from tqdm import tqdm
for dataset in tqdm(["ISOT", "pan2020"]):#)""LIAR_PANTS","pan2020", "ISOT", "AAAI2021_COVID19_fake_news", "FakeNewsNet"]):
    export_LM(dataset)

  0%|          | 0/2 [00:00<?, ?it/s]18-Mar-21 21:26:43 - Load pretrained SentenceTransformer: roberta-large-nli-stsb-mean-tokens
18-Mar-21 21:26:43 - Did not find folder roberta-large-nli-stsb-mean-tokens
18-Mar-21 21:26:43 - Try to download model from server: https://sbert.net/models/roberta-large-nli-stsb-mean-tokens.zip
18-Mar-21 21:26:43 - Load SentenceTransformer from folder: /home/boshkok/.cache/torch/sentence_transformers/sbert.net_models_roberta-large-nli-stsb-mean-tokens
18-Mar-21 21:26:48 - Use pytorch device: cuda
